# 📘 Agentic Architectures 2：Tool Use

本章讨论第二种、也是最具变革性的 agentic architecture 之一：**Tool Use**。这个模式是连接 Large Language Model（LLM）的推理能力与真实动态世界的桥梁。

没有工具时，LLM 是一个封闭系统，被训练数据中冻结的知识所限制。它无法知道今天的天气、某只股票的当前价格，或你公司数据库中某个订单的状态。通过赋予 agent 使用工具的能力，我们让它越过这一根本限制：可以调用 API、查询数据库、访问实时信息，从而给出不仅有推理、而且具备事实性、时效性与相关性的答案。

### 定义
**Tool Use** 架构让一个由 LLM 驱动的 agent 能够调用外部函数或 API（即“工具”）。当用户的问题无法仅靠其内部知识回答时，agent 会自主判断并决定调用哪个合适的工具以获取所需信息。

### 高层流程

1.  **接收查询：** agent 接收用户请求。
2.  **决策：** agent 分析查询与可用工具，判断是否需要工具才能准确回答。
3.  **行动：** 若需要工具，agent 会格式化一次工具调用（例如以正确参数调用某函数）。
4.  **观察：** 系统执行工具调用，并将结果（“observation”）返回给 agent。
5.  **综合：** agent 将工具的输出整合进其推理过程，生成最终、可落地的答案。

### 何时使用 / 应用场景
*   **研究助理：** 通过 web search API 获取最新信息来回答问题。
*   **企业助理：** 查询公司内部数据库，回答诸如“上周新增了多少用户？”之类的问题。
*   **科学与数学任务：** 使用 calculator 或像 WolframAlpha 这样的计算引擎来进行 LLM 常常不擅长的精确计算。

### 优势与劣势
*   **优势：**
    *   **事实性支撑：** 通过获取真实、实时数据，大幅降低幻觉。
    *   **可扩展性：** 仅需不断添加新工具即可持续扩展 agent 能力。
*   **劣势：**
    *   **集成成本：** 需要正确的“管线”以定义工具、处理 API keys，并管理可能的工具失败。
    *   **工具可靠性：** agent 的答案质量依赖于所用工具的可靠性与准确性，agent 需要信任其工具提供正确信息。

## 阶段 0：基础与配置

与之前一样，我们先完成环境配置。这包括安装必要的库，并为 Nebius、LangSmith 以及本 notebook 将使用的具体工具配置 API keys。

### 步骤 0.1：安装核心库

**我们要做什么：**
将安装用于编排的 `langchain-nebius`、`langgraph`，用于环境管理的 `python-dotenv`，以及用于打印的 `rich`。此外，我们还将安装 `tavily-python`，它提供便捷的 web search API，作为我们赋予 agent 的工具。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv tavily-python

### 步骤 0.2：导入库并配置 Keys

**我们要做什么：**
导入所需模块，并用 `python-dotenv` 加载 API keys。本 notebook 需要 Nebius（用于 LLM）、LangSmith（用于 tracing）以及 Tavily（用于 web search 工具）的 keys。

**需要操作：** 在当前目录创建 `.env` 并写入：
````
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
````

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel, Field

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tool Use (Nebius)"

# Check that the keys are set
for key in ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 阶段 1：定义 Agent 的工具箱

一个 agent 的能力取决于其可用工具。在此阶段，我们将定义并测试将赋予 agent 的具体工具：实时 web search。

### 步骤 1.1：创建并测试 Web Search 工具

**我们要做什么：**
实例化 `TavilySearchResults` 工具。定义工具的**描述**至关重要。LLM 依赖这段自然语言描述理解工具用途与使用时机。描述需要清晰、准确，才能使 agent 做出正确决策。随后我们将直接测试该工具，观察其原始输出。

In [3]:
# Initialize the tool. We can set the max number of results to keep the context concise.
search_tool = TavilySearchResults(max_results=2)

# It's crucial to give the tool a clear name and description for the agent
search_tool.name = "web_search"
search_tool.description = "A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs."

tools = [search_tool]
print(f"Tool '{search_tool.name}' created with description: '{search_tool.description}'")

console = Console()

# Let's test the tool directly to see its output format
print("\n--- Testing the tool directly ---")
test_query = "What was the score of the last Super Bowl?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query:[/bold green] {test_query}")
console.print("\n[bold green]Result:[/bold green]")
console.print(test_result)

Tool 'web_search' created with description: 'A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs.'

--- Testing the tool directly ---


C:\Users\faree\AppData\Local\Temp\ipykernel_2556\1362582966.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


Query: What was the score of the last Super Bowl?

Result:

[
    {
        'title': 'List of Super Bowl Winners (1967-2025) - NFL Champions by Year Last 10 Super Bowls Scores | 
StatMuse Super Bowl Winners by Year: Complete List & 2025 Results Super Bowl Winners by Year - ESPN List of Super 
Bowl Winners (1967-2025) - NFL Champions by Year List of Super Bowl champions - Wikipedia Super Bowl Winners by 
Year - ESPN Super Bowl Winners by Year: Complete List & 2025 Results List of Super Bowl champions - Wikipedia List 
of Super Bowl Winners (1967-2025) - NFL Champions by Year Who Played in The Super Bowl Last Year? - Bleacher 
Nation',
        'url': 'https://www.topendsports.com/events/super-bowl/winners-list.htm',
        'content': 'Score: Rams 23, Bengals 20\n Venue: SoFi Stadium, Los Angeles\n Date: February 13, 2022\n MVP: 
Cooper Kupp\n\n### Super Bowls LV & LIV\n\n2021 (LV): Tampa Bay Buccaneers defeated Kansas City Chiefs 31-9, with 
Tom Brady winning his 7th championship.\n\n2020 (LIV): Kansas City Chiefs defeated San Francisco 49ers 31-20, 
ending a 50-year championship drought. [...] | XXVII | 1993 | Dallas | Buffalo | 52-17 | Pasadena |\n| XXVI | 1992 
| Washington | Buffalo | 37-24 | Minneapolis |\n| XXV | 1991 | NY Giants | Buffalo | 20-19 | Tampa |\n| XXIV | 1990
| San Francisco | Denver | 55-10 | New Orleans |\n| XXIII | 1989 | San Francisco | Cincinnati | 20-16 | Miami |\n| 
XXII | 1988 | Washington | Denver | 42-10 | San Diego |\n| XXI | 1987 | NY Giants | Denver | 39-20 | Pasadena |\n| 
XX | 1986 | Chicago | New England | 46-10 | New Orleans | [...] | No. | Year | Winner | Opposition | Score | Venue 
|\n| LIX | 2025 | Philadelphia Eagles | Kansas City Chiefs | 40-22 | New Orleans, Louisiana |\n| LVIII | 2024 | 
Kansas City Chiefs | San Francisco 49ers | 25-22 | Las Vegas, Nevada |\n| LVII | 2023 | Kansas City Chiefs | 
Philadelphia Eagles | 38-35 | Arizona |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23-20 | Los Angeles
|\n| LV | 2021 | Tampa Bay Buccaneers | Kansas City Chiefs | 31-9 | Tampa |',
        'score': 0.7300017
    },
    {
        'title': 'Super Bowl Winners by Year: Complete List & 2025 Results',
        'url': 'https://nflplayoffpass.com/super-bowl-winners/',
        'content': '| Super Bowl | Year | Winner | Opposition | Score | Stadium |\n ---  ---  --- |\n| LIX | 2025 |
Philadelphia Eagles | Kansas City Chiefs | 40-22 | Caesars Superdome |\n| LVIII | 2024 | Kansas City Chiefs | San 
Francisco 49ers | 25–22 (OT) | Allegiant Stadium |\n| LVII | 2023 | Kansas City Chiefs | Philadelphia Eagles | 
38–35 | State Farm Stadium |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23–20 | SoFi Stadium | [...] |
XLV | 2011 | Green Bay Packers | Pittsburgh Steelers | 31–25 | Cowboys Stadium |\n| XLIV | 2010 | New Orleans 
Saints | Indianapolis Colts | 31–17 | Sun Life Stadium |\n| XLIII | 2009 | Pittsburgh Steelers | Arizona Cardinals 
| 27–23 | Raymond James Stadium |\n| XLII | 2008 | New York Giants | New England Patriots | 17–14 | University of 
Phoenix Stadium |\n| XLI | 2007 | Indianapolis Colts | Chicago Bears | 29–17 | Dolphin Stadium | [...] | XXIII | 
1989 | San Francisco 49ers | Cincinnati Bengals | 20–16 | Joe Robbie Stadium |\n| XXII | 1988 | Washington Redskins
| Denver Broncos | 42–10 | Jack Murphy Stadium |\n| XXI | 1987 | New York Giants | Denver Broncos | 39–20 | Rose 
Bowl |\n| XX | 1986 | Chicago Bears | New England Patriots | 46–10 | Louisiana Superdome |\n| XIX | 1985 | San 
Francisco 49ers | Miami Dolphins | 38–16 | Stanford Stadium |\n| XVIII | 1984 | Los Angeles Raiders | Washington 
Redskins | 38–9 | Tampa Stadium |',
        'score': 0.7293082
    }
]

**输出讨论：**
测试展示了 `web_search` 工具的原始输出。它返回一个字典列表，每个字典包含搜索结果的 URL 与内容片段。这类结构化信息正是 agent 在决定使用工具后，作为“observation”接收到的内容。完成工具验证后，我们可以构建会使用该工具的 agent。

## 阶段 2：用 LangGraph 构建会使用工具的 Agent

现在我们来构建 agent 的工作流。关键在于让 LLM 具备工具意识，并用图让它循环执行“思考-行动-观察”的闭环，这正是 Tool Use 的精髓。

### 步骤 2.1：定义图的状态

**我们要做什么：**
一个使用工具的 agent 的状态通常是一组 messages，表示会话历史。这段历史包括用户问题、agent 的思考与工具调用，以及工具返回的结果。我们将使用可容纳任意 LangChain message 的 `TypedDict`。

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

print("AgentState TypedDict defined to manage conversation history.")

AgentState TypedDict defined to manage conversation history.


### 步骤 2.2：将工具绑定到 LLM

**我们要做什么：**
这是让 LLM 具备“工具意识”的关键步骤。使用 `.bind_tools()` 方法，把工具的名称和描述写入 LLM 的系统提示，让模型能够基于描述在内部逻辑中决定何时调用对应工具。

In [5]:
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)

# Bind the tools to the LLM, making it tool-aware
llm_with_tools = llm.bind_tools(tools)

print("LLM has been bound with the provided tools.")

LLM has been bound with the provided tools.


### 步骤 2.3：定义 Agent 节点

**我们要做什么：**
图包含两个主要节点：
1.  **`agent_node`：** 相当于“脑”，用当前会话历史调用 LLM。其响应要么是最终答案，要么是工具调用请求。
2.  **`tool_node`：** 相当于“手”，接收来自 `agent_node` 的工具调用请求，执行工具并返回输出。这里我们使用 LangGraph 的预置 `ToolNode`。

In [6]:
def agent_node(state: AgentState):
    """The primary node that calls the LLM to decide the next action."""
    console.print("--- AGENT: Thinking... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# The ToolNode is a pre-built node from LangGraph that executes tools
tool_node = ToolNode(tools)

print("Agent node and Tool node have been defined.")

Agent node and Tool node have been defined.


### 步骤 2.4：定义条件路由

**我们要做什么：**
在 `agent_node` 之后，我们需要决定下一步去向。路由函数会检查 agent 的最后一条 message。若该 message 包含 `tool_calls` 属性，说明 agent 希望使用工具，于是路由到 `tool_node`；否则说明 agent 已给出最终回答，可以结束流程。

In [7]:
def router_function(state: AgentState) -> str:
    """Inspects the agent's last message to decide the next step."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # The agent has requested a tool call
        console.print("--- ROUTER: Decision is to call a tool. ---")
        return "call_tool"
    else:
        # The agent has provided a final answer
        console.print("--- ROUTER: Decision is to finish. ---")
        return "__end__"

print("Router function defined.")

Router function defined.


## 阶段 3：组装并运行工作流

现在我们把所有组件接入，形成一个完整、可执行的图，并用一个问题来强迫 agent 使用其新获得的 web search 能力。

### 步骤 3.1：构建与可视化图

**我们要做什么：**
创建 `StateGraph`，添加节点与边。关键是使用 `router_function` 的条件边来形成核心推理循环：`agent -> router -> tool -> agent`。

In [8]:
graph_builder = StateGraph(AgentState)

# Add the nodes
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("call_tool", tool_node)

# Set the entry point
graph_builder.set_entry_point("agent")

# Add the conditional router
graph_builder.add_conditional_edges(
    "agent",
    router_function,
)

# Add the edge from the tool node back to the agent to complete the loop
graph_builder.add_edge("call_tool", "agent")

# Compile the graph
tool_agent_app = graph_builder.compile()

print("Tool-using agent graph compiled successfully!")

# Visualize the graph
try:
    from IPython.display import Image, display
    png_image = tool_agent_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Tool-using agent graph compiled successfully!
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


**输出讨论：**
图已编译完成。可视化清晰展示了 agent 的推理循环：流程从 `agent` 节点开始，经由菱形的条件路由后，若需要工具则进入 `call_tool`，其输出再反馈给 `agent` 进行综合；若不需要工具则进入 `__end__`。该结构完整实现了 Tool Use 模式。

### 步骤 3.2：端到端执行

**我们要做什么：**
运行一个训练数据中不可能已知的问题，迫使 agent 使用 web search 工具。我们将以流式方式输出中间步骤，观察推理过程如何展开。

In [9]:
user_query = "What were the main announcements from Apple's latest WWDC event?"
initial_input = {"messages": [("user", user_query)]}

console.print(f"[bold cyan]🚀 Kicking off Tool Use workflow for request:[/bold cyan] '{user_query}'\n")

for chunk in tool_agent_app.stream(initial_input, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("\n[bold green]✅ Tool Use workflow complete![/bold green]")

🚀 Kicking off Tool Use workflow for request: 'What were the main announcements from Apple's latest WWDC event?'

================================ Human Message =================================

What were the main announcements from Apple's latest WWDC event?


---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

...

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

================================== Ai Message ==================================

The main announcements from Apple's latest WWDC event include a new design that will inform the next decade of iOS, iPadOS, and macOS development, new features for the iPhone, an overhauled Spotlight interface for the Mac, and updates that make the iPad more like a Mac than ever before. Additionally, Apple introduced a ton of new features and updates across every platform, including iOS 26, iPadOS 26, CarPlay, macOS Tahoe, watchOS 26, visionOS 26, and tvOS 26.


---

✅ Tool Use workflow complete!

## 阶段 4：评估

运行完成后我们来评估其表现。对于会使用工具的 agent，我们关注两点：是否正确使用工具，以及最终综合工具输出得到的答案是否高质量。

### 步骤 4.1：分析执行轨迹

**我们要做什么：**
从上一步的流式输出中可追踪 agent 的完整思路。输出展示了在图状态中流动的不同消息类型（包含 `tool_calls` 的 `AIMessage`、带结果的 `ToolMessage`）。

**输出讨论：**
轨迹清楚展示了 Tool Use 的执行方式：
1.  第一条打印来自 `agent` 节点，是带 `tool_calls` 的 `AIMessage`，说明 LLM 正确决定使用 `web_search`。
2.  随后是 `ToolMessage`，这是 `tool_node` 执行搜索并返回的原始结果。
3.  最后再次是 `AIMessage`，但不再有 `tool_calls`，这是 agent 将 `ToolMessage` 的信息综合为用户的最终答案。
该轨迹验证了 agent 的逻辑与图路由的正确性。

### 步骤 4.2：用 LLM-as-a-Judge 进行量化评估

**我们要做什么：**
创建一个“Judge” LLM，给出结构化、定量的评估指标，专门衡量工具使用质量。

In [10]:
class ToolUseEvaluation(BaseModel):
    """Schema for evaluating the agent's tool use and final answer."""
    tool_selection_score: int = Field(description="Score 1-5 on whether the agent chose the correct tool for the task.")
    tool_input_score: int = Field(description="Score 1-5 on how well-formed and relevant the input to the tool was.")
    synthesis_quality_score: int = Field(description="Score 1-5 on how well the agent integrated the tool's output into its final answer.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# To evaluate, we need to reconstruct the full conversation trace
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.content or ''} {getattr(m, 'tool_calls', '')}" for m in final_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""You are an expert judge of AI agents. Evaluate the following conversation trace based on the agent's tool use on a scale of 1-5. Provide a brief justification.
    
    Conversation Trace:
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Tool Use Performance ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

--- Evaluating Tool Use Performance ---

{
    'tool_selection_score': 5,
    'tool_input_score': 5,
    'synthesis_quality_score': 4,
    'justification': "The AI agent used the web search tool to find relevant information about Apple's latest WWDC 
event. The tool output was well-formed and relevant, providing a good summary of the announcements. However, the AI
agent could have done a better job of synthesizing the information, as some of the points mentioned in the output 
are not clearly connected to the main announcements. For example, the mention of the Apple Watch and other devices 
seems out of place in the context of the main announcements. Overall, the AI agent's tool use was effective, but 
could be improved with better synthesis of the information."
}

**输出讨论：**
LLM-as-a-Judge 给出了结构化且有理据的评估。三个维度——`tool_selection_score`、`tool_input_score` 与 `synthesis_quality_score` 的较高分表明我们的 agent 不仅“会用工具”，而且“用得有效”。它正确识别需要 web search，构造了相关查询，并将检索到的事实成功综合为有帮助且准确的最终答案。这类自动化评估让我们对实现的稳健性更有信心。

## 总结

在本 notebook 中，我们基于 **Tool Use** 架构搭建了一个完整、可运行的 agent。我们成功赋予了一个由 Nebius 驱动的 LLM 以 web search 工具的能力，并用 LangGraph 创建了稳健的推理闭环，让 agent 能够自主决定何时、如何使用工具。

端到端执行与后续评估展示了该模式的巨大价值。通过连接外部实时信息，我们从根本上突破了静态训练数据的限制。agent 不再只是一个推理器，它还是一个研究者，能够提供具有事实性与时效性的答案。该架构为构建几乎任何实用的、面向现实世界的 AI 助理奠定了基础。